<a href="https://colab.research.google.com/github/ingabLee/Book_GenerateAIForComptureVision/blob/main/GenerateAI_Vision_Chapter_2_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

In [5]:
import os
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
from google.colab import drive

drive.mount('/content/drive')

os.makedirs('/content/drive/MyDrive/AE_img', exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def normalization(tensor, min_value, max_value):
  min_tensor = tensor.min()
  tensor = (tensor - min_tensor)
  max_tensor = tensor.max()
  tensor = tensor / max_tensor
  tensor = tensor * (max_value - min_value) + min_value
  return tensor

def value_round(tensor):
  return torch.round(tensor)

